# Nonlinear Benchmark Workshop Datasets

> Fill in a module description here

In [ ]:
#| default_exp workshop

In [ ]:
#| export
from sysbench_loader.core import *
import nonlinear_benchmarks
from pathlib import Path

In [ ]:
#| hide
import shutil

In [ ]:
tmp_dir = Path('./tmp')

## Wiener Hammerstein Dataset

In [ ]:
#| export
def wiener_hammerstein(
        save_path: Path #directory the files are written to, created if it does not exist
):
    save_path = save_path / 'wh'
    train_val, test = nonlinear_benchmarks.WienerHammerBenchMark()
    split_idx = 80_000
    train = train_val[:split_idx]
    valid = train_val[split_idx:]

    dataset_to_hdf5(train,valid,test,save_path / 'wh')

In [ ]:
wiener_hammerstein(tmp_dir)

## Silverbox Dataset

In [ ]:
#| export
def silverbox(
        save_path: Path #directory the files are written to, created if it does not exist
):
    save_path = save_path / 'silverbox'
    train_val, test = nonlinear_benchmarks.Silverbox()
    split_idx = 78_000
    train = train_val[:split_idx]
    valid = train_val[split_idx:]

    dataset_to_hdf5(train,valid,test,save_path / 'silverbox')

In [ ]:
silverbox(tmp_dir)

## Cascaded Tanks Dataset

In [ ]:
#| export
def cascaded_tanks(
        save_path: Path #directory the files are written to, created if it does not exist
):
    'the cascaded_tanks dataset, '
    save_path = save_path / 'cascaded_tanks'
    train_val, test = nonlinear_benchmarks.Cascaded_Tanks()
    train = train_val
    valid = train_val

    dataset_to_hdf5(train,valid,test,save_path / 'cascaded_tanks')

In [ ]:
cascaded_tanks(tmp_dir)

## EMPS Dataset

In [ ]:
#| export
def emps(
        save_path: Path #directory the files are written to, created if it does not exist
):
    'the emps dataset, '
    save_path = save_path / 'emps'
    train_val, test = nonlinear_benchmarks.EMPS()
    split_idx = 18_000
    train = train_val[:split_idx]
    valid = train_val[split_idx:]

    dataset_to_hdf5(train,valid,test,save_path / 'emps')

In [ ]:
emps(tmp_dir)

## Noisy Wiener Hammerstein

In [ ]:
# nonlinear_benchmarks.not_splitted_benchmarks.WienerHammerstein_Process_Noise()

dataset not found downloading from https://data.4tu.nl/file/1f194001-affa-4459-870a-ad9e9d9146f9/2dbbc046-1ac2-43b2-bf4e-53b5a4be8b96 
 in C:\Users\danie\AppData\Local\nonlinear_benchmarks/WienHammer\WienerHammersteinFiles.zip


In [ ]:
#| hide
#clean temporary hdf5 file
# shutil.rmtree(tmp_dir)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()